In [14]:
import tensorflow as tf
print(tf.__version__)
import torch
print(torch.__version__)

from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSeq2SeqLM,
   AdamW
)

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, Trainer, TrainingArguments

import speech_recognition as sr
from gtts import gTTS
import os
import time
import numpy as np

2.15.0
2.1.1+cu118


In [10]:


class ChatBot():
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("C:/Users/alvar/Desktop/UCM-TFM-G1/code/DialoGPT-medium")
        self.model = AutoModelForCausalLM.from_pretrained("C:/Users/alvar/Desktop/UCM-TFM-G1/code/DialoGPT-medium")

    def get_input(self):
        user_choice = input("Choose input type (text/speech): ").lower()
        if user_choice == "text":
            user_input = input("You: ")
        elif user_choice == "speech":
            user_input = self.speech_to_text()
        else:
            print("Invalid choice. Please choose 'text' or 'speech'.")
            user_input = self.get_input()
        return user_input

    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
        try:
            text = recognizer.recognize_google(audio)
            print("Speech-to-Text --> ", text)
            return text
        except sr.UnknownValueError:
            print("Speech-to-Text --> Could not understand audio")
            return "ERROR"

    @staticmethod
    def text_to_speech(text):
        print("ChatBot: ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        os.system('start res.mp3')
        time.sleep(2)
        os.remove("res.mp3")

    def generate_response(self, input_text, past_user_inputs, generated_responses):
        # Encode the conversation history and user input
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512)

        # Generate a response
        with torch.no_grad():
            response_ids = self.model.generate(input_ids)
        
        # Decode and return the response
        generated_text = self.tokenizer.decode(response_ids[0], skip_special_tokens=True)
        return generated_text

if __name__ == "__main__":
    ai = ChatBot()
    
    ex = True
    past_user_inputs = []
    generated_responses = []
    
    while ex:
        user_input = ai.get_input()
        
        # exit condition
        if user_input.lower() == "exit":
            ex = False
            continue

        # process input and generate response
        response = ai.generate_response(user_input, past_user_inputs, generated_responses)
        ai.text_to_speech(response)
        
        # update conversation history
        past_user_inputs.append(user_input)
        generated_responses.append(response)

    print("----- Closing down Dev -----")

Choose input type (text/speech): when does winter start?
Invalid choice. Please choose 'text' or 'speech'.
Choose input type (text/speech): text
You: when does winter start?


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\alvar\anaconda3\lib\site-packages\transformers\generation\utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ChatBot:  when does winter start?


KeyboardInterrupt: Interrupted by user

In [4]:
# model_name = "microsoft/DialoGPT-large"
model_name = "microsoft/DialoGPT-large"
# model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# chatting 5 times with nucleus sampling & tweaking temperature
for step in range(5):
    # take user input
    text = input(">> You:")
    # encode the input and add end of string token
    input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")
    # concatenate new user input with chat history (if there is)
    bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1) if step > 0 else input_ids
    # generate a bot response
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
    #print the output
    output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Waste ChatBot: {output}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

...

Waste ChatBot: Why would you think it was?
>> You:You look like a Gallego answering with an answer


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Waste ChatBot: Do you know how to recycle?
>> You:better than you, idiot


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Waste ChatBot: And you look like a girl, not a gallego.


In [16]:
pip install accelerate

In [18]:
pip install transformers[torch]

In [19]:
# Load the GPT-2 model and tokenizer
model_name = "microsoft/DialoGPT-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config)

# Define your training arguments and trainer
training_args = TrainingArguments(
    output_dir="./waste_classification_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Train the model (assuming train_dataset is your fine-tuning dataset)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

# Use the fine-tuned model for chatbot responses
for step in range(5):
    text = input(">> You:")
    input_ids = tokenizer.encode("Waste Bro is here to help. " + text + tokenizer.eos_token, return_tensors="pt")
    bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1) if step > 0 else input_ids
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        do_sample=True,
        top_p=0.95,
        top_k=0,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"Waste Bro: {output}")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`